<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intellignce"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.33.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.33.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [ ]:
// A new decision prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

Here are five decision-making frameworks that can enhance the quality of decisions:

---

### 1. **SWOT Analysis**
   - **Description:** SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework is used to evaluate the internal and external factors influencing a decision.
   - **How it supports better decisions:**
     - Encourages a structured exploration of both positive and negative factors.
     - Helps identify opportunities and risks that may not be immediately obvious.
     - Provides a holistic view to better align actions with organizational objectives or personal goals.

---

### 2. **Cost-Benefit Analysis (CBA)**
   - **Description:** CBA involves evaluating all the potential costs and benefits of a decision to determine if the benefits outweigh the costs.
   - **How it supports better decisions:**
     - Quantifies pros and cons for an objective comparison.
     - Encourages a focus on long-term outcomes rather than short-term gains.
     - Useful fo

### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> Note: An average human can read between 25-40 Tokens / second.  

In [ ]:
// Same Decision Prompt but using Streaming Output 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Here are five decision-making frameworks, along with brief descriptions of how they enhance analysis and reasoning across various scenarios:

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**
   - **Description:** Originally designed for military strategy by Col. John Boyd, the OODA loop helps decision-makers quickly assess situations and adapt to changing circumstances.
   - **How it helps:**
     - **Observe:** Encourages thorough situational awareness by gathering data and identifying key factors.
     - **Orient:** Promotes analysis by reconciling new information with existing knowledge and context.
     - **Decide:** Emphasizes focused decision-making to choose the best course of action.
     - **Act:** Stresses execution and learning from outcomes, making the framework iterative and adaptive.
   - **Applications:** Emergency responses, competitive business strategy, and crisis management.

---

### 2. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **

### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can ouput Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting ouptut. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [5]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Do not enclose the table in triple backticks.
""";

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptWithMarkdownFormat))
{
   Console.Write(streamChunk);
}

| Decision-Making Framework | Description                                                                                             |
|----------------------------|---------------------------------------------------------------------------------------------------------|
| SWOT Analysis             | Evaluates Strengths, Weaknesses, Opportunities, and Threats of a situation to improve strategic clarity. |
| Cost-Benefit Analysis     | Weighs the pros and cons by quantifying outcomes, enabling data-driven decisions.                       |
| Decision Matrix           | Uses weighted criteria to compare multiple options systematically and rank them objectively.            |
| Six Thinking Hats         | Encourages analyzing problems from various perspectives (emotional, logical, creative, etc.) to ensure balanced decisions. |
| OODA Loop                 | Stands for Observe, Orient, Decide, Act; enables iterative decision-making in dynamic and uncertain environments. |

Example formatted Markdown Table output: 

| Decision-Making Framework           | Description                                                                                   |
|-------------------------------------|-----------------------------------------------------------------------------------------------|
| **Rational Decision-Making Model** | A step-by-step process focusing on identifying the problem, gathering data, and evaluating alternatives based on logic, ensuring well-informed and objective decisions. |
| **SWOT Analysis**                   | Helps systematically assess Strengths, Weaknesses, Opportunities, and Threats, offering a structured way to evaluate strategic options. |
| **Decision Tree Analysis**          | Visualizes choices and their possible outcomes using a tree diagram, aiding in weighing different paths based on probabilities and payoffs. |
| **Cost-Benefit Analysis (CBA)**     | Quantifies costs and benefits of alternatives, enabling comparison to identify the option with the highest net benefit.                 |
| **OODA Loop (Observe-Orient-Decide-Act)** | A dynamic, iterative model aiding in rapid decision-making by continuously assessing and adapting to changing scenarios.              |

### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

> Note: The supported settings are dependendent on the API plus the specific model version. For example, a model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available.  

In [6]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Here are five decision-making frameworks that can enhance the quality of decisions, along with brief descriptions of how each supports better analysis and reasoning:

---

### 1. **Rational Decision-Making Model**
   - **Description**: This framework involves a structured, step-by-step process to identify the problem, gather information, evaluate alternatives, and choose the best option based on logical reasoning.
   - **How it helps**: By emphasizing data collection, objective analysis, and systematic evaluation, this model minimizes biases and ensures decisions are grounded in evidence and logic. It is particularly useful in scenarios requiring detailed analysis, such as business planning or policy development.

---

### 2. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT analysis is a strategic tool used to evaluate internal and external factors that impact a decision or situation.
   - **How it helps**: By categorizing factors into streng

### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decison Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [ ]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("RESPONSE FROM THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);

// Stream the response asynchronously
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...

System Prompt: 
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.

Request from the user: 
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.

----------------------------------------

RESPONSE FROM THE CHAT COMPLETION SERVICE...

Here are five simple yet powerful decision-making frameworks that can be applied to a wide range of daily situations. Each is designed to be easy to understand and implement, while still providing structure and clarity to your decision-making process.

---

### **1. The Eisenhower Matrix (Urgent-Important Matrix)**
**Purpose:** Prioritize tasks and decisions based on urgency and importance.


### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full Decision Intelligence framework will not be used, rather a simple request for Artificial Intelligene to recommend a path forward (recommedation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [8]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed decision scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommedation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(scenarioDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

### Recommendation for Alex: Start at a Community College

Based on the provided decision factors, starting at a community college and then transferring to a four-year university is the most balanced and strategic choice for Alex. Below is a structured breakdown of the reasoning:

| **Decision Factor**            | **Community College**                                                                                     | **Four-Year University**                                                                                   | **Recommendation**                                                                                          |
|--------------------------------|-----------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------